___

# [ Machine Learning in Geosciences ] 
Department of Applied Geoinformatics and Carthography, Charles University

Lukas Brodsky lukas.brodsky@natur.cuni.cz


Thi notebook demonstrates rasterization and vectorization (polygomnaztion) processe with Python GDAL/OGR

* 1. Rasterization of polygons; 

* 2. Vectorization of classification results; 


### Rasterization

In [ ]:
import os
import sys
import matplotlib.pyplot as plt 
%matplotlib inline 
from osgeo import gdal
from osgeo import ogr
from osgeo import osr

#### Data

In [ ]:
# Set your own PATH!!! 
PATH = './data/'

if os.path.isdir(PATH): 
    print('ok')

In [ ]:
raster_fn = os.path.join(PATH, 'landsat.tif')
vector_fn = os.path.join(PATH, 'polygons.shp')

In [ ]:
os.path.isfile(raster_fn)

In [ ]:
os.path.isfile(vector_fn)

#### Reading raster and meatadata 

In [ ]:
ds = gdal.Open(raster_fn)
geo_transform = ds.GetGeoTransform()
# geotransform metadata
x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * ds.RasterXSize
y_min = y_max + geo_transform[5] * ds.RasterYSize
pixel_width = geo_transform[1]
# array dimensionality
x_size = ds.RasterXSize
y_size = ds.RasterYSize

#### Rasterization

In [ ]:
# filename 
output_fn = os.path.join(PATH, 'rasterized_poygons.tif')

In [ ]:
# define pixel_size and NoData value of new raster
pixel_size = 30
NoData_value = -99

# open the data source and read in the extent
source_ds = ogr.Open(vector_fn)
source_layer = source_ds.GetLayer()

# create the destination data source
target_ds = gdal.GetDriverByName('GTiff').Create(output_fn, x_size, y_size, 1, gdal.GDT_Byte)
target_ds.SetGeoTransform(geo_transform)
spatialRef = source_layer.GetSpatialRef()
projection = spatialRef.ExportToWkt()
target_ds.SetProjection(projection)
band = target_ds.GetRasterBand(1)
band.SetNoDataValue(NoData_value)

# rasterize
gdal.RasterizeLayer(target_ds, [1], source_layer, options=["ATTRIBUTE=label"])

In [ ]:
# save it! 
source_ds = None
target_ds = None

In [ ]:
# Check in QGIS! 

### Raster vectorization

In [ ]:
ndsi_fn = os.path.join(PATH, 'landsat_ndsi.tif')
os.path.isfile(ndsi_fn)

In [ ]:
ndsi_ds = gdal.Open(ndsi_fn)
ndsi_band = ndsi_ds.GetRasterBand(1)
ndsi = ndsi_band.ReadAsArray()

In [ ]:
plt.imshow(ndsi, interpolation='none', cmap='RdBu', vmin=-0.3, vmax=0.05)
plt.colorbar()

#### Raster classification map

In [ ]:
glacier = ndsi > -0.1

In [ ]:
glacier

In [ ]:
plt.imshow(glacier, cmap='RdBu')

In [ ]:
# metadata
geo_transform = ndsi_ds.GetGeoTransform()
# print(geo_transform)
projection = ndsi_ds.GetProjectionRef()
# print(projection)

In [ ]:
# first save the raster
filename = os.path.join(PATH, 'glacier_classification.tif')

In [ ]:
target_ds = gdal.GetDriverByName('GTiff').Create(filename, x_size, y_size, 1, gdal.GDT_Byte)
target_ds.SetGeoTransform(geo_transform)
target_ds.SetProjection(projection)

In [ ]:
ndsi_band = target_ds.GetRasterBand(1).WriteArray(glacier * 1)
target_ds = None

In [ ]:
cls_ds = gdal.Open(filename)
cls_band = cls_ds.GetRasterBand(1)

In [ ]:
cls_band

In [ ]:
plt.imshow(cls_band.ReadAsArray())

#### Prepare verctor

In [ ]:
# vector fine name
vector_fn = os.path.join(PATH, 'glacier_classification.shp')

In [ ]:
def prepare_vector(fn, raster_fn): 
    driver = ogr.GetDriverByName('ESRI Shapefile')
    # prepare data source to be cloused to save the results! 
    out_data = driver.CreateDataSource(fn)
    layer_name = os.path.basename(fn).split('.')[0]
    ndsi_ds = gdal.Open(raster_fn)
    srs = osr.SpatialReference()
    srs.ImportFromWkt(ndsi_ds.GetProjectionRef())
    out_layer = out_data.CreateLayer(layer_name, srs)
    fld = ogr.FieldDefn("class", ogr.OFTInteger)
    out_layer.CreateField(fld)
    dst_field = out_layer.GetLayerDefn().GetFieldIndex("class")
    
    return out_data, out_layer, dst_field

In [ ]:
out_data, out_layer, dst_field = prepare_vector(vector_fn, ndsi_fn)

#### Polygonize

In [ ]:
cls_band

In [ ]:
# http://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html#polygonize-a-raster-band
gdal.Polygonize(cls_band, cls_band, out_layer, dst_field, ["8CONNECTED=8"], callback=None)
# not yeat saved to file! 

In [ ]:
# closing 
out_data = None

cls_ds = None
ndsi_ds = None